In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from common import get_cfg, prepare_val
import numpy as np
import cv2
import matplotlib.pyplot as plt

DATASET_DIR = '/media/hcis-s20/SRL/nuscenes/trainval/'
vehicle_idx = [4, 5, 6, 7, 8, 10, 11,12]
DYNAMIC = [
    'car', 'truck', 'bus',
    'trailer', 'construction_vehicle',
    'pedestrian',
    'motorcycle', 'bicycle',
    # 'emergency',
]
torch.autograd.set_detect_anomaly(True)

In [2]:
version = 'cvt_labels_nuscenes_v2'
cfg1 = get_cfg(DATASET_DIR, version, 'SparseBEVSeg_Det_3') # cvt_nuscenes_multiclass Sparse
device = torch.device('cuda:0') # cuda:5
model_version = '2024_0826_180130'
CHECKPOINT_PATH = None
# CHECKPOINT_PATH = '../logs/cross_view_transformers_test/' + model_version +'/checkpoints/last.ckpt'
model, network, loader, viz, dataset = prepare_val(cfg1, device, CHECKPOINT_PATH, mode='val', batch_size=2)

/media/hcis-s20/SRL/cross_view_ae/cross_view_transformers/scripts/common.py:50: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path='../config')
/media/hcis-s20/SRL/cross_view_ae/cross_view_transformers/scripts/common.py:14: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path='./config')


Loaded pretrained weights for efficientnet-b4
Dataset length: 6019


In [3]:
network.to(device)
model.to(device)
model.metrics.reset()
with_grad = True

if with_grad:
    network.train()
    # with torch.no_grad():
    for i,batch in enumerate(loader):
        print(i,end='\r')
        if i != 0:
            continue
        for k, v in batch.items():
            if isinstance(v, torch.Tensor):
                batch[k] = v.to(device)
            elif isinstance(v, list):
                if isinstance(v[0], torch.Tensor):
                    batch[k] = [i.to(device) for i in v]
            else:
                batch[k] = v
        pred = network(batch)
        loss = model.loss_func(pred,batch)
        loss[0].backward()
        vis_result = viz(batch,pred)
        break
else:
    network.eval()
    with torch.no_grad():
        for i,batch in enumerate(loader):
            print(i,end='\r')
            for k, v in batch.items():
                if isinstance(v, torch.Tensor):
                    batch[k] = v.to(device)
                elif isinstance(v, list):
                    if isinstance(v[0], torch.Tensor):
                        batch[k] = [i.to(device) for i in v]
                else:
                    batch[k] = v
            pred = network(batch)
            # pred['mid_output']['sampled_feats'].retain_grad()
            loss = model.loss_func(pred,batch)
            # loss[0].backward()
            vis_result = viz(batch,pred)
    # if i ==5:
            break   
        
if CHECKPOINT_PATH is None:
    for name, param in model.named_parameters():
        if param.requires_grad and param.grad is None:
            print(name)
model.metrics.update(pred,batch)


In [ ]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=128)
# pts = pred['mid_output']['sampled_feats'][0].reshape(-1,2,128).sigmoid()
# row, column = 1, 2
# fig, ax = plt.subplots(row, column, figsize=(10, 4))
# for i in range(2):
#     feat = pts[:,i]#.reshape(-1, 128)
#     feat_ = pca.fit_transform(feat).reshape(200,200,128)
#     ax[i].imshow(feat_[:,:,:50].sum(-1))
    # ax[i//column, i%column].imshow(feat_[:,:,:1])
# feat = pred['height']['sampled_feat'][0,128:256].permute(1,2,0).reshape(-1, 128)
# feat_ = pca.fit_transform(feat).reshape(200,200,128)
# plt.imshow(feat_[:,:,:1])

In [ ]:
batch_index = 0
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8))
y = 20
x = 97
vehicle_idx = [[4,5,6,7,8,10,11]]
vehicles = [batch['bev'][:, idx].max(1, keepdim=True).values for idx in vehicle_idx]
vehicles = torch.cat(vehicles, 1)
ax1.imshow(vehicles[batch_index,0].detach().cpu().numpy())
ax1.scatter([x],[y], s=2, c='r')
ax1.set_title('gt')

ax2.imshow(((pred['VEHICLE'][batch_index,0].sigmoid().detach().cpu())).numpy())
ax2.scatter([x],[y], s=2, c='r')
ax2.set_title('Prediction')
# plt.title('2D')

In [ ]:
# mean height supervised
x = [[[0.2583, 0.5916]],

        [[0.2611, 0.5548]],

        [[0.2590, 0.5263]],

        [[0.2577, 0.4973]],

        [[0.2594, 0.4648]],

        [[0.2635, 0.4029]],

        [[0.2552, 0.3652]],

        [[0.2559, 0.3101]]]

y = [[[0.2996, 0.5774]],

        [[0.3006, 0.5427]],

        [[0.2995, 0.5147]],

        [[0.2988, 0.4870]],

        [[0.2990, 0.4578]],

        [[0.3013, 0.3992]],

        [[0.2969, 0.3625]],

        [[0.2972, 0.3111]]]

z = [[[0.3370, 0.5645]],

        [[0.3387, 0.5318]],

        [[0.3373, 0.5054]],

        [[0.3364, 0.4787]],

        [[0.3373, 0.4501]],

        [[0.3396, 0.3953]],

        [[0.3343, 0.3607]],

        [[0.3348, 0.3138]]]

x = np.array(x)[:,0]
x[:, 0] *= 480
x[:, 1] *= 224

y = np.array(y)[:,0]
y[:, 0] *= 480
y[:, 1] *= 224

z = np.array(z)[:,0]
z[:, 0] *= 480
z[:, 1] *= 224

plt.imshow(batch['image'][0,2].cpu().permute(1,2,0))
plt.scatter(x[:,0],x[:,1],s=1, c='b')
plt.scatter(y[:,0],y[:,1],s=1, c='g')
plt.scatter(z[:,0],z[:,1],s=1, c='r')
plt.axis('off')
plt.show()

In [ ]:
plt.imshow(vis_result[0])

In [ ]:
from nuscenes.nuscenes import NuScenes

# Load the nuScenes dataset (mini-split, in this case).
nusc = NuScenes(version='v1.0-trainval', dataroot='/media/hcis-s20/SRL/nuscenes/trainval/', verbose=False)

In [ ]:
for batch in loader:
    break
nusc.render_sample(batch['token'][0])

In [ ]:
# viz.label_indices = [4, 5, 6, 7,8,9,10,11]
# from cross_view_transformer.visualizations.common import get_colors
# SEMANTICS = [viz.SEMANTICS[i] for i in viz.label_indices]
# viz.colors = get_colors(SEMANTICS)
for batch in loader:
    break
plt.imshow(viz.visualize_bev(batch['bev'][0], batch['view'][0].numpy()))
# plt.legend(loc="upper right", bbox_to_anchor=(1.25, 1.0))
plt.axis('off')

In [ ]:
plt.imshow(viz(batch)[0])

In [ ]:
a = torch.rand((4, 40000, 1, 8, 128))
a = a.reshape(4 *40000,8, 128)
attention = nn.MultiheadAttention(128, 4, batch_first=True)
attention(a, a, a)

In [ ]:
batch = dataset.__getitem__(4004)
for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        batch[k] = v[None]
# with torch.no_grad():
#     pred = network(batch)
pred = None
vis_result = viz(batch, pred)
plt.imshow(vis_result[0])

In [ ]:
plt.imshow(pred['VEHICLE'][0,0].sigmoid())

In [ ]:
a = np.random.rand((5))
print(a)

In [ ]:
(a > 0.6).all()

TypeError: list indices must be integers or slices, not list